**Pytorch BERT baseline**

In this version, I convert https://www.kaggle.com/akensert/bert-base-tf2-0-minimalistic into pytorch version

**Please upvote the kernel if you find it helpful**

As we are not allowed to use internet I've created required datasets and commands to setup Hugging Face Transformers setup in offline mode. You can find the required github codebases in the datasets.

* sacremoses dependency - https://www.kaggle.com/axel81/sacremoses
* transformers - https://www.kaggle.com/axel81/transformers

In [1]:
import random
import re
import pandas as pd
from nltk import sent_tokenize
from tqdm import tqdm
from albumentations.core.transforms_interface import DualTransform, BasicTransform
# !pip install ./sacremoses/sacremoses-master/
# !pip install ./transformers/transformers-master/

STRIDE = 1
def is_jupyter():
    try:
        ipy_str = str(type(get_ipython()))
        if 'zmqshell' in ipy_str:
            return True
        
    except:
        return False

### Required Imports

I've added imports that will be used in training too

In [2]:
import re
from random import shuffle as shfl
from auc import MyAUCCallback
from sklearn.utils import shuffle
from datetime import datetime
import pandas as pd
pd.set_option('display.max.columns', 500)
import numpy as np
import os
# os.environ['CUDA_VISIBLE_DEVICES'] ='1'
# os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda-10.1/lib64'
import matplotlib.pyplot as plt
import gc
from shutil import copyfile
from catalyst.dl import SupervisedRunner, AlchemyLogger, CriterionCallback
from catalyst.dl.callbacks.metrics import AUCCallback
from torch.utils.data import DataLoader, SubsetRandomSampler,Dataset
batch_size =96
token = "d1dd16f08d518293bcbeddd313b49aa4"
DATA_DIR = '/kaggle/input/jigsaw-multilingual-toxic-comment-classification/'

/home/kb/jig_env/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
/home/kb/jig_env/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/home/kb/jig_env/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject



In [3]:
if os.uname()[1] == 'kb-Z370P-D3':
    # desktop
    LOG_PATH = '/media/ssd/logs/jigsaw'
    SERVER = False
    print('Working on desktop')
elif os.uname()[1] == 'kb-server':
    # server
    LOG_PATH = '/home/kb/logs/jigsaw'
    SERVER = True
    print('Working on server')
else:
    raise Exception('which hostname???')
    

Working on desktop


In [4]:
def clean(text):
    text = text.fillna("fillna")#.str.lower()
    text = text.map(lambda x: re.sub('\\n',' ',str(x)))
    text = text.map(lambda x: re.sub("\[\[User.*",'',str(x)))
    text = text.map(lambda x: re.sub("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}",'',str(x)))
    text = text.map(lambda x: re.sub("\(http://.*?\s\(http://.*\)",'',str(x)))
    return text

puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£',
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', '\xa0', '\t',
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '\u3000', '\u202f',
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '«',
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

mispell_dict = {"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"couldnt" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"doesnt" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"havent" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"shouldnt" : "should not",
"that's" : "that is",
"thats" : "that is",
"there's" : "there is",
"theres" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"theyre":  "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not",
"tryin'":"trying"}

from nltk.tokenize.treebank import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()

def handle_contractions(x):
    x = tokenizer.tokenize(x)
    return x

def fix_quote(x):
    x = [x_[1:] if x_.startswith("'") else x_ for x_ in x]
    x = ' '.join(x)
    return x

def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re


def replace_typical_misspell(text):
    mispellings, mispellings_re = _get_mispell(mispell_dict)

    def replace(match):
        return mispellings[match.group(0)]

    return mispellings_re.sub(replace, text)

def clean_text(x):
    x = str(x).replace("\n","")
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x


def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

def clean_data(df, columns: list):
    for col in columns:
        pass
        df[col] = df[col].apply(lambda x: clean_numbers(x))
        df[col] = clean(df[col])
        df[col] = df[col].apply(lambda x: clean_text(x)) 
        df[col] = df[col].apply(lambda x: replace_typical_misspell(x))
# #         df[col] = df[col].apply(lambda x: handle_contractions(x))  
#         df[col] = df[col].apply(lambda x: fix_quote(x))   
    
    return df

<>:4: DeprecationWarning:

invalid escape sequence \[

<>:5: DeprecationWarning:

invalid escape sequence \d

<>:6: DeprecationWarning:

invalid escape sequence \(



In [5]:
class NLPTransform(BasicTransform):
    """ Transform for nlp task."""
    LANGS = {
        'en': 'english',
        'it': 'italian', 
        'fr': 'french', 
        'es': 'spanish',
        'tr': 'turkish', 
        'ru': 'russian',
        'pt': 'portuguese'
    }

    @property
    def targets(self):
        return {"data": self.apply}
    
    def update_params(self, params, **kwargs):
        if hasattr(self, "interpolation"):
            params["interpolation"] = self.interpolation
        if hasattr(self, "fill_value"):
            params["fill_value"] = self.fill_value
        return params

    def get_sentences(self, text, lang='en'):
        return sent_tokenize(text, self.LANGS.get(lang, 'english'))

In [6]:
class ShuffleSentencesTransform(NLPTransform):
    """ Do shuffle by sentence """
    def __init__(self, always_apply=False, p=0.5):
        super(ShuffleSentencesTransform, self).__init__(always_apply, p)

    def apply(self, data, **params):
        text, lang = data
        sentences = self.get_sentences(text, lang)
        random.shuffle(sentences)
        return ' '.join(sentences), lang
    
transform = ShuffleSentencesTransform(p=1.0)

text = '<Sentence1>. <Sentence2>. <Sentence3>. <Sentence4>. <Sentence5>. <Sentence6>.'
lang = 'en'

transform(data=(text, lang))['data'][0]

'<Sentence4>. <Sentence1>. <Sentence5>. <Sentence2>. <Sentence3>. <Sentence6>.'

In [7]:
class ExcludeDuplicateSentencesTransform(NLPTransform):
    """ Exclude equal sentences """
    def __init__(self, always_apply=False, p=0.5):
        super(ExcludeDuplicateSentencesTransform, self).__init__(always_apply, p)

    def apply(self, data, **params):
        text, lang = data
        sentences = []
        for sentence in self.get_sentences(text, lang):
            sentence = sentence.strip()
            if sentence not in sentences:
                sentences.append(sentence)
        return ' '.join(sentences), lang
transform = ExcludeDuplicateSentencesTransform(p=1.0)

text = '<Sentence1>. <Sentence2>. <Sentence4>. <Sentence4>. <Sentence5>. <Sentence5>.'
lang = 'en'

transform(data=(text, lang))['data'][0]

'<Sentence1>. <Sentence2>. <Sentence4>. <Sentence5>.'

In [8]:
class ExcludeNumbersTransform(NLPTransform):
    """ exclude any numbers """
    def __init__(self, always_apply=False, p=0.5):
        super(ExcludeNumbersTransform, self).__init__(always_apply, p)

    def apply(self, data, **params):
        text, lang = data
        text = re.sub(r'[0-9]', '', text)
        text = re.sub(r'\s+', ' ', text)
        return text, lang
transform = ExcludeNumbersTransform(p=1.0)

text = '<Word1> <Word2> <Word3> <Word4> <Word5> <Word6> <Word7> <Word8> <Word9> <Word10>'
lang = 'en'

transform(data=(text, lang))['data'][0]

'<Word> <Word> <Word> <Word> <Word> <Word> <Word> <Word> <Word> <Word>'

In [9]:
class ExcludeHashtagsTransform(NLPTransform):
    """ Exclude any hashtags with # """
    def __init__(self, always_apply=False, p=0.5):
        super(ExcludeHashtagsTransform, self).__init__(always_apply, p)

    def apply(self, data, **params):
        text, lang = data
        text = re.sub(r'#[\S]+\b', '', text)
        text = re.sub(r'\s+', ' ', text)
        return text, lang
transform = ExcludeHashtagsTransform(p=1.0)

text = '<Word1> <Word2> <Word3> #kaggle <Word4> <Word5> <Word6> <Word7> <Word8> <Word9> <Word10>'
lang = 'en'

transform(data=(text, lang))['data'][0]

'<Word1> <Word2> <Word3> <Word4> <Word5> <Word6> <Word7> <Word8> <Word9> <Word10>'

In [10]:
class ExcludeUsersMentionedTransform(NLPTransform):
    """ Exclude @users """
    def __init__(self, always_apply=False, p=0.5):
        super(ExcludeUsersMentionedTransform, self).__init__(always_apply, p)

    def apply(self, data, **params):
        text, lang = data
        text = re.sub(r'@[\S]+\b', '', text)
        text = re.sub(r'\s+', ' ', text)
        return text, lang
transform = ExcludeUsersMentionedTransform(p=1.0)

text = '<Word1> <Word2> <Word3> @kaggle <Word4> <Word5> <Word6> <Word7> <Word8> <Word9> <Word10>'
lang = 'en'

transform(data=(text, lang))['data'][0]

'<Word1> <Word2> <Word3> <Word4> <Word5> <Word6> <Word7> <Word8> <Word9> <Word10>'

In [11]:
class ExcludeUrlsTransform(NLPTransform):
    """ Exclude urls """
    def __init__(self, always_apply=False, p=0.5):
        super(ExcludeUrlsTransform, self).__init__(always_apply, p)

    def apply(self, data, **params):
        text, lang = data
        text = re.sub(r'https?\S+', '', text)
        text = re.sub(r'\s+', ' ', text)
        return text, lang
transform = ExcludeUrlsTransform(p=1.0)

text = '<Word1> <Word2> <Word3> <Word4> https://www.kaggle.com/shonenkov/nlp-albumentations/ <Word6> <Word7> <Word8> <Word9> <Word10>'
lang = 'en'

transform(data=(text, lang))['data'][0]

'<Word1> <Word2> <Word3> <Word4> <Word6> <Word7> <Word8> <Word9> <Word10>'

In [12]:
class SwapWordsTransform(NLPTransform):
    """ Swap words next to each other """
    def __init__(self, swap_distance=1, swap_probability=0.1, always_apply=False, p=0.5):
        """  
        swap_distance - distance for swapping words
        swap_probability - probability of swapping for one word
        """
        super(SwapWordsTransform, self).__init__(always_apply, p)
        self.swap_distance = swap_distance
        self.swap_probability = swap_probability
        self.swap_range_list = list(range(1, swap_distance+1))

    def apply(self, data, **params):
        text, lang = data
        words = text.split()
        words_count = len(words)
        if words_count <= 1:
            return text, lang

        new_words = {}
        for i in range(words_count):
            if random.random() > self.swap_probability:
                new_words[i] = words[i]
                continue
    
            if i < self.swap_distance:
                new_words[i] = words[i]
                continue
    
            swap_idx = i - random.choice(self.swap_range_list)
            new_words[i] = new_words[swap_idx]
            new_words[swap_idx] = words[i]

        return ' '.join([v for k, v in sorted(new_words.items(), key=lambda x: x[0])]), lang

transform = SwapWordsTransform(p=1.0, swap_distance=1, swap_probability=0.2)

text = '<Word1> <Word2> <Word3> <Word4> <Word5> <Word6> <Word7> <Word8> <Word9> <Word10>'
lang = 'en'

transform(data=(text, lang))['data'][0]

'<Word1> <Word2> <Word3> <Word4> <Word6> <Word5> <Word7> <Word8> <Word9> <Word10>'

In [13]:
class CutOutWordsTransform(NLPTransform):
    """ Remove random words """
    def __init__(self, cutout_probability=0.05, always_apply=False, p=0.5):
        super(CutOutWordsTransform, self).__init__(always_apply, p)
        self.cutout_probability = cutout_probability

    def apply(self, data, **params):
        text, lang = data
        words = text.split()
        words_count = len(words)
        if words_count <= 1:
            return text, lang
        
        new_words = []
        for i in range(words_count):
            if random.random() < self.cutout_probability:
                continue
            new_words.append(words[i])

        if len(new_words) == 0:
            return words[random.randint(0, words_count-1)], lang

        return ' '.join(new_words), lang
transform = CutOutWordsTransform(p=1.0, cutout_probability=0.2)

text = '<Word1> <Word2> <Word3> <Word4> <Word5> <Word6> <Word7> <Word8> <Word9> <Word10>'
lang = 'en'

transform(data=(text, lang))['data'][0]

'<Word1> <Word2> <Word3> <Word4> <Word6> <Word7> <Word8> <Word9> <Word10>'

In [14]:
class AddNonToxicSentencesTransform(NLPTransform):
    """ Add random non toxic statement """
    def __init__(self, non_toxic_sentences, sentence_range=(1, 3), always_apply=False, p=0.5):
        super(AddNonToxicSentencesTransform, self).__init__(always_apply, p)
        self.sentence_range = sentence_range
        self.non_toxic_sentences = non_toxic_sentences

    def apply(self, data, **params):
        text, lang = data

        sentences = self.get_sentences(text, lang)
        for i in range(random.randint(*self.sentence_range)):
            sentences.append(random.choice(self.non_toxic_sentences))
        
        random.shuffle(sentences)
        return ' '.join(sentences), lang


In [15]:
nlp_transform = NLPTransform()

df = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv', nrows=1000)
df = df[df.toxic == 0]
df['lang'] = 'en'
non_toxic_sentences = set()
for comment_text in tqdm(df['comment_text'], total=df.shape[0]):
    non_toxic_sentences.update(nlp_transform.get_sentences(comment_text), 'en')

transform = AddNonToxicSentencesTransform(non_toxic_sentences=list(non_toxic_sentences), p=1.0, sentence_range=(1,2))

100%|██████████| 895/895 [00:00<00:00, 7591.82it/s]


In [16]:
import albumentations

def get_train_transforms():
    return albumentations.Compose([
        ExcludeDuplicateSentencesTransform(p=0.9),  # here not p=1.0 because your nets should get some difficulties
        albumentations.OneOf([
            AddNonToxicSentencesTransform(non_toxic_sentences=list(non_toxic_sentences), p=0.8, sentence_range=(1,3)),
            ShuffleSentencesTransform(p=0.8),
        ]),
        ExcludeNumbersTransform(p=0.8),
        ExcludeHashtagsTransform(p=0.5),
        ExcludeUsersMentionedTransform(p=0.9),
        ExcludeUrlsTransform(p=0.9),
        CutOutWordsTransform(p=0.1),
        SwapWordsTransform(p=0.1),
    ])

In [17]:
train1 = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv")
train2 = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv")
train2.toxic = train2.toxic.round().astype(int)

df_valid = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv')
sub = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv')

In [18]:
# Combine train1 with a subset of train2
df_train = pd.concat([
    train1[['comment_text', 'toxic']],
    train2[['comment_text', 'toxic']].query('toxic==1'),
    train2[['comment_text', 'toxic']].query('toxic==0').sample(n=100000, random_state=0)
])

In [19]:
df_valid_en = shuffle(pd.read_csv(DATA_DIR + 'validation_en.csv'))
df_valid_en = df_valid_en.drop(['comment_text'],axis=1).rename(columns={'comment_text_en':'comment_text'})
# df_valid_en = clean_data(df_valid_en, ['comment_text'])
df_valid_en.head()

,id,lang,toxic,comment_text
640,640,tr,0,"For some reason I changed as a source, the Tur..."
6902,6902,tr,0,There is a problem in the exclusive pattern of...
1554,1554,it,0,I read but mostly Madaki is always arrogant an...
6664,6664,es,0,Brutes are characterized as very fools enemies...
626,626,tr,1,"Gay men in gay men, gay women women. Is that ..."


In [20]:
df_valid_en.groupby(['lang', 'toxic']).count()

id  comment_text
lang toxic                    
es   0      2078          2078
     1       422           422
it   0      2012          2012
     1       488           488
tr   0      2680          2680
     1       320           320

In [21]:
# df_test = pd.read_csv(DATA_DIR + 'test.csv')
# df_test.head()

In [22]:
# df_test.groupby(['lang']).count()

In [23]:
# len(df_test)

In [24]:
df_sub = pd.read_csv(DATA_DIR + 'sample_submission.csv')
df_sub.head()

,id,toxic
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [25]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [26]:
target_column = 'toxic'

### Define dataset

In [27]:
# 7from scipy.stats import spearmanr
from sklearn.model_selection import train_test_split, cross_val_score, KFold
import time
from torch.optim import lr_scheduler

import torch
from tqdm import tqdm
#import torch.utils.data as data
from torchvision import datasets, models, transforms
from transformers import *
import random
from math import floor, ceil
from sklearn.model_selection import GroupKFold

MAX_LEN = 96#192#192#512
SEP_TOKEN_ID = 102

class QuestDataset(torch.utils.data.Dataset):
    def __init__(self, df, train_mode=True, labeled=True, train_transforms=None):
        
        self.train_transforms = train_transforms    
        self.df = df
        if train_mode:
            self.labels = df.toxic.values
            self.toxic_inds = np.where(self.labels==1)[0]
            self.normal_inds = np.where(self.labels==0)[0]            
            
            
            print(f'Here is {len(self.labels)} samples, {len(self.toxic_inds)} samples and {len(self.normal_inds)} samples')
            print(f'Class balance is {len(self.toxic_inds)/len(self.labels):.2f}')
            
        self.train_mode = train_mode
        self.labeled = labeled
#         self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#         self.tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
#         self.tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
#         self.tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
        self.tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-large')#, 
#                                                              return_attention_masks=False, 
#                                                                 return_token_type_ids=False,
#                                                                 pad_to_max_length=True,
#                                                                 max_length=MAX_LEN)
#         self.tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased',
#                                                                 do_lower_case=False,
#                                                                 do_basic_tokenize=True,
#                                                                 never_split=None,
#                                                                 unk_token='[UNK]',
#                                                                 sep_token='[SEP]',
#                                                                 pad_token='[PAD]',
#                                                                 cls_token='[CLS]',
#                                                                 mask_token='[MASK]',
#                                                                 tokenize_chinese_chars=True,)
        #distil

    def __getitem__(self, index):
        row = self.df.iloc[index]
        token_ids = self.get_token_ids(row)
        
        if self.labeled:
            labels = self.get_label(row)
            return {'features': token_ids, 'targets': labels}

        else:
            return {'features': token_ids}

    def __len__(self):
        return len(self.df)

    def trim_input(self, text, max_sequence_length=MAX_LEN):
        t = self.tokenizer.tokenize(text)
        t_len = len(t)

        if t_len + 2 > max_sequence_length:

            t_new_len = int(max_sequence_length) - 2

            t = t[:t_new_len]

        return t
        
    def get_token_ids(self, row):
        text = row.comment_text
        if self.train_transforms:
            lang = 'en' if 'lang' not in list(row.keys().values) else row.lang
            text, _ = self.train_transforms(data=(row.comment_text, lang))['data']
        token_ids = self.tokenizer.encode(text, max_length=1024)
        if len(token_ids) < MAX_LEN:
            ids = torch.tensor(token_ids + [0] * (MAX_LEN - len(token_ids)))
        else:
            ind_beg = random.randint(0, len(token_ids)-MAX_LEN)
            ids = torch.tensor(token_ids[ind_beg:ind_beg+MAX_LEN])
            
        return ids

    def get_label(self, row):
#         label = torch.tensor(row[target_column].astype(np.long))
        label = np.round(row[target_column])
        return torch.tensor([1-label, label]).float()
    
    def collate_fn(self, batch):
        token_ids = torch.stack([x[0] for x in batch])

        if self.labeled:
            labels = torch.stack([x[1] for x in batch])
            return {'features': token_ids, 'targets': labels}
        else:
            return {'features': token_ids}
        
 

## Build Model

In [28]:
from transformers import *
import torch
import torch.nn as nn
import torch.nn.functional as F


class QuestModel(nn.Module):
    def __init__(self, n_classes=2):
        super(QuestModel, self).__init__()
        self.model_name = 'QuestModel'
        
#         self.bert_model = BertModel.from_pretrained('bert-base-uncased') 
#         self.bert_model = DistilBertModel.from_pretrained('distilbert-base-uncased')
#         self.bert_model = DistilBertModel.from_pretrained('distilbert-base-multilingual-cased')
#         self.bert_model = BertModel.from_pretrained('bert-base-multilingual-cased')
#        self.bert_model = XLMRobertaModel.from_pretrained('xlm-roberta-base', 
                                                          #output_hidden_states=False, 
                                                          #output_attentions=False)
#         self.bert_model = DistilBertModel.from_pretrained('distilbert-base-cased')
        self.bert_model = XLMRobertaModel.from_pretrained('xlm-roberta-large')#,
#                                                           output_hidden_states=False, 
#                                                           output_attentions=False)
    
#         self.fc = nn.Linear(768, n_classes)
        self.fc = nn.Linear(1024, n_classes)

    def forward(self, ids):
#         attention_mask = (ids > 0)
#         print(ids.shape)
        layers = self.bert_model(input_ids=ids)#, attention_mask=attention_mask)
#         print(layers[0].shape)
#         out = F.dropout(layers[0][:, 0, :], p=0.2, training=self.training)
#         print(layers[0].shape)
#         print([l.shape for l in layers])
#         out = F.dropout(layers[-1][:, 0, :], p=0.35, training=self.training)
        out = F.dropout(layers[0][:, 0, :], p=0.2, training=self.training)
        logit = self.fc(out)#.unsqueeze(1)
        return logit #, 'for_auc': logit[:, 1]}#[:,1]
    


In [29]:
# bert_model = XLMRobertaModel.from_pretrained('xlm-roberta-base')

In [30]:
class BalancedSampler(torch.utils.data.sampler.Sampler):
    def __init__(self, dataset):
       
        self.toxic_inds = dataset.toxic_inds.copy()
        self.normal_inds = dataset.normal_inds.copy()
        
        self.num_samples = 2*min(len(self.toxic_inds), len(self.normal_inds))
        
        shfl(self.toxic_inds)
        shfl(self.normal_inds)
        
        self.inds = []
        for i in range(min(len(self.toxic_inds), len(self.normal_inds))):
            self.inds.append(self.normal_inds[i%len(self.normal_inds)])
            self.inds.append(self.toxic_inds[i%len(self.toxic_inds)])

    def __iter__(self):
        #print ('\tcalling Sampler:__iter__')
        return iter(self.inds)

    def __len__(self):
        #print ('\tcalling Sampler:__len__')
        return self.num_samples




In [31]:
class ImbalancedDatasetSampler(torch.utils.data.sampler.Sampler):
    """Samples elements randomly from a given list of indices for imbalanced dataset
    Arguments:
        indices (list, optional): a list of indices
        num_samples (int, optional): number of samples to draw
        callback_get_label func: a callback-like function which takes two arguments - dataset and index
    """

    def __init__(self, dataset, indices=None, num_samples=None, callback_get_label=None):
                
        # if indices is not provided, 
        # all elements in the dataset will be considered
        self.indices = list(range(len(dataset))) \
            if indices is None else indices

        # define custom callback
        self.callback_get_label = callback_get_label

        # if num_samples is not provided, 
        # draw `len(indices)` samples in each iteration
        self.num_samples = len(self.indices) \
            if num_samples is None else num_samples
            
        # distribution of classes in the dataset 
        label_to_count = {}
        for idx in self.indices:
            label = self._get_label(dataset, idx)
            if label in label_to_count:
                label_to_count[label] += 1
            else:
                label_to_count[label] = 1
                
        # weight for each sample
        weights = [1.0 / label_to_count[self._get_label(dataset, idx)]
                   for idx in self.indices]
        self.weights = torch.DoubleTensor(weights)

    def _get_label(self, dataset, idx):
        if self.callback_get_label:
            return self.callback_get_label(dataset, idx)
        else:
            dataset.labels[idx]
#             raise NotImplementedError
                
    def __iter__(self):
        return (self.indices[i] for i in torch.multinomial(
            self.weights, self.num_samples, replacement=True))

    def __len__(self):
        return self.num_samples

In [32]:
def get_loaders(to_balance=True, shuffle_before=True):
    if SERVER:
        workers=1
    else:
        workers = 6    
    
#     train_dataset = QuestDataset(df_train.iloc[::STRIDE], train_mode=True)
    
    if STRIDE == 1:
        train_dataset = QuestDataset(df_train.iloc[::STRIDE], train_mode=True, train_transforms=get_train_transforms())
    elif STRIDE<=10:
        df_train_pos = df_train[df_train[target_column]==1].reset_index(drop=True)
        df_train_neg = df_train[df_train[target_column]==0].reset_index(drop=True).iloc[::STRIDE]
        train_dataset = QuestDataset(df_train_pos.append(df_train_neg).reset_index(drop=True), train_mode=True, train_transforms=get_train_transforms())
    else:
        df_train_pos = df_train[df_train[target_column]==1].reset_index(drop=True).iloc[::int(STRIDE/10)]
        df_train_neg = df_train[df_train[target_column]==0].reset_index(drop=True).iloc[::STRIDE]
        train_dataset = QuestDataset(df_train_pos.append(df_train_neg).reset_index(drop=True), train_mode=True, train_transforms=get_train_transforms())
        
    
    valid_dataset = QuestDataset(df_valid, train_mode=False)
   
    
    train_loader = DataLoader(
        train_dataset,
        num_workers=workers,
        sampler=BalancedSampler(train_dataset) if to_balance else None,#ImbalancedDatasetSampler(train_dataset) if to_balance else None,
        batch_size=batch_size,
    )
    valid_loader = DataLoader(
        valid_dataset,
        num_workers=workers,
        batch_size=batch_size,
    )    
    
       
    loaders = {}
    loaders['train'] = train_loader
    
    loaders['valid'] = valid_loader
    
    
    for i in ['es', 'it', 'tr']:
        df = df_valid
        df = df[df['lang']==i]

        loaders['valid_'+ i] = DataLoader(
            QuestDataset(df, train_mode=False),
            num_workers=workers,
            batch_size=batch_size,
        )
    
    
    return loaders


In [33]:
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [34]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, logits=False, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.logits = logits
        self.reduce = reduce

    def forward(self, inputs, targets):
        if self.logits:
            BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduce=False)
        else:
            BCE_loss = F.binary_cross_entropy(inputs, targets, reduce=False)
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss

In [35]:
def freeze_until(net, param_name):
    found_name = False
    for name, params in net.named_parameters():
        if name == param_name:
            found_name = True
        params.requires_grad = found_name
param_name = 'bert_model.encoder.layer.10.attention.self.query.weight'

CHECKPOINT = './checkpoints/v93_xlm_roberta_large_ml96/best.pth'
project = "jigsaw_v94_xlm_roberta_large_fp02_layer10fr"
num_epochs = 300

group = datetime.now().strftime("%m_%d_%Y__%H_%M_%S")

    
if SERVER:
    group = f'{group}_server'
    
if STRIDE > 1:
    group = f'{group}_str{STRIDE}'


gradient_accumulation_steps = 1

lr = 5e-6#1e-5#0.0001
group += f'_lr{lr}_dr0.2_base_toofp16_96_myaug'

group = group.replace('.', '')

runner = SupervisedRunner(input_key=('features'), input_target_key=('targets'), output_key=('logits'))

experiment = 'simple'

torch.cuda.empty_cache()

logdir = f"{LOG_PATH}/{project}/{group}/{experiment}"

model = QuestModel(2)
freeze_until(model, param_name)
#     checkpoint = torch.load(CHECKPOINT)#, map_location=device)   

#     model.load_state_dict(checkpoint['model_state_dict'])
#     del checkpoint
# model = model.to(device)


loaders = get_loaders(to_balance=True)


t_total = len(loaders['train'])//gradient_accumulation_steps*num_epochs
warmup_proportion = 0.01
num_warmup_steps = t_total * warmup_proportion

#     criterion = torch.nn.BCEWithLogitsLoss()
criterion = FocalLoss(alpha=0.2, gamma=1.5, logits=True, reduce=True)
#     criterion = torch.nn.CrossEntropyLoss()
#     criterion = torch.nn.BCELoss()
optimizer = AdamW(model.parameters(), lr = lr)
#    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=t_total) 
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.25)
print(f'----------------Experiment: {experiment}')

runner.train(
    fp16=dict(opt_level="O2") ,
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=loaders,
    logdir=logdir,
    num_epochs=num_epochs,
    verbose=True,
    distributed=False if is_jupyter() else True,
    callbacks=[
        AlchemyLogger(
                token=token, # your Alchemy token
                project=project,
                experiment=experiment,
                group=group,
            ),
        MyAUCCallback()

    ],
    main_metric='auc/_mean',
    minimize_metric=False,
    
    #fp16={"opt_level": "O1"}

)

Here is 435775 samples, 133610 samples and 302165 samples
Class balance is 0.31
----------------Experiment: simple
Selected optimization level O2:  FP16 training with FP32 batchnorm and FP32 master weights.

Defaults for this optimization level are:
enabled                : True
opt_level              : O2
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : True
master_weights         : True
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O2
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : True
master_weights         : True
loss_scale             : dynamic
1/300 * Epoch (train):   0% 0/2784 [00:00<?, ?it/s]

/home/kb/jig_env/lib/python3.7/site-packages/torch/nn/_reduction.py:43: UserWarning:

size_average and reduce args will be deprecated, please use reduction='none' instead.



1/300 * Epoch (train):   0% 1/2784 [00:01<47:00,  1.01s/it, loss=0.063]

/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning:

This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)



1/300 * Epoch (train):   2% 58/2784 [00:22<17:35,  2.58it/s, loss=0.053]Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
1/300 * Epoch (train):   3% 84/2784 [00:32<17:30,  2.57it/s, loss=0.042]Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
1/300 * Epoch (train): 100% 2784/2784 [18:07<00:00,  2.56it/s, loss=0.016]
1/300 * Epoch (valid): 100% 84/84 [00:14<00:00,  5.73it/s, loss=0.016]
1/300 * Epoch (valid_es): 100% 27/27 [00:04<00:00,  5.52it/s, loss=0.007]
1/300 * Epoch (valid_it): 100% 27/27 [00:04<00:00,  5.55it/s, loss=0.007]
1/300 * Epoch (valid_tr): 100% 32/32 [00:05<00:00,  5.47it/s, loss=0.003]
[2020-05-04 00:15:22,038] 
1/300 * Epoch 1 (_base): lr=5.000e-06 | momentum=0.9000
1/300 * Epoch 1 (train): auc/_mean=0.9351 | auc/class_0=0.9351 | loss=0.0238
1/300 * Epoch 1 (valid): auc/_mean=0.9031 | auc/class_0=0.9031 | es_auc/_mean=0.8805 | es_auc/class_0=0.8805 | es_loss=0.0297 | it_auc/_mean=0.8581 | it_auc/class_0=0.8

8/300 * Epoch (train): 100% 2784/2784 [18:08<00:00,  2.56it/s, loss=0.012]
8/300 * Epoch (valid): 100% 84/84 [00:14<00:00,  5.74it/s, loss=0.021]
8/300 * Epoch (valid_es): 100% 27/27 [00:04<00:00,  5.47it/s, loss=0.004]
8/300 * Epoch (valid_it): 100% 27/27 [00:04<00:00,  5.46it/s, loss=0.008]
8/300 * Epoch (valid_tr): 100% 32/32 [00:05<00:00,  5.54it/s, loss=0.011]
[2020-05-04 02:29:14,960] 
8/300 * Epoch 8 (_base): lr=5.000e-06 | momentum=0.9000
8/300 * Epoch 8 (train): auc/_mean=0.9797 | auc/class_0=0.9797 | loss=0.0134
8/300 * Epoch 8 (valid): auc/_mean=0.8920 | auc/class_0=0.8920 | es_auc/_mean=0.8624 | es_auc/class_0=0.8624 | es_loss=0.0290 | it_auc/_mean=0.8411 | it_auc/class_0=0.8411 | it_loss=0.0346 | loss=0.0266 | tr_auc/_mean=0.9621 | tr_auc/class_0=0.9621 | tr_loss=0.0165
8/300 * Epoch 8 (valid_es): auc/_mean=0.8624 | auc/class_0=0.8624 | loss=0.0290
8/300 * Epoch 8 (valid_it): auc/_mean=0.8411 | auc/class_0=0.8411 | loss=0.0346
8/300 * Epoch 8 (valid_tr): auc/_mean=0.9621 |

15/300 * Epoch (train):  35% 973/2784 [06:19<11:46,  2.56it/s, loss=0.007]Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 131072.0
15/300 * Epoch (train):  91% 2536/2784 [16:30<01:36,  2.56it/s, loss=0.012]Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 65536.0
15/300 * Epoch (train): 100% 2784/2784 [18:07<00:00,  2.56it/s, loss=0.003]
15/300 * Epoch (valid): 100% 84/84 [00:14<00:00,  5.75it/s, loss=0.018]
15/300 * Epoch (valid_es): 100% 27/27 [00:04<00:00,  5.48it/s, loss=0.001]
15/300 * Epoch (valid_it): 100% 27/27 [00:04<00:00,  5.50it/s, loss=0.011]
15/300 * Epoch (valid_tr): 100% 32/32 [00:05<00:00,  5.50it/s, loss=0.011]
[2020-05-04 04:42:18,033] 
15/300 * Epoch 15 (_base): lr=5.000e-06 | momentum=0.9000
15/300 * Epoch 15 (train): auc/_mean=0.9893 | auc/class_0=0.9893 | loss=0.0098
15/300 * Epoch 15 (valid): auc/_mean=0.8740 | auc/class_0=0.8740 | es_auc/_mean=0.8464 | es_auc/class_0=0.8464 | es_loss=0.0396 | it_auc/_mean=0.8233 | i

22/300 * Epoch (train):  61% 1707/2784 [11:06<07:00,  2.56it/s, loss=0.005]Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 65536.0
22/300 * Epoch (train):  98% 2730/2784 [17:45<00:21,  2.56it/s, loss=0.010]Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
22/300 * Epoch (train): 100% 2784/2784 [18:06<00:00,  2.56it/s, loss=0.002]
22/300 * Epoch (valid): 100% 84/84 [00:14<00:00,  5.73it/s, loss=0.019]
22/300 * Epoch (valid_es): 100% 27/27 [00:04<00:00,  5.51it/s, loss=0.001]
22/300 * Epoch (valid_it): 100% 27/27 [00:04<00:00,  5.48it/s, loss=0.009]
22/300 * Epoch (valid_tr): 100% 32/32 [00:05<00:00,  5.47it/s, loss=0.012]
[2020-05-04 06:55:10,631] 
22/300 * Epoch 22 (_base): lr=5.000e-06 | momentum=0.9000
22/300 * Epoch 22 (train): auc/_mean=0.9931 | auc/class_0=0.9931 | loss=0.0078
22/300 * Epoch 22 (valid): auc/_mean=0.8680 | auc/class_0=0.8680 | es_auc/_mean=0.8382 | es_auc/class_0=0.8382 | es_loss=0.0478 | it_auc/_mean=0.8120 | i

Early exiting                                                             
29/300 * Epoch (train):  14% 390/2784 [02:32<15:36,  2.56it/s, loss=0.002]

In [36]:
# for name, params in model.named_parameters():
#     print(name)

#### 

In [67]:
# from apex import amp
df_tst = pd.read_csv('/media/ssd/Downloads/submission.csv')
df_tst.head()

,id,toxic
0,0,0.146779
1,1,0.216815
2,2,0.656836
3,3,0.065074
4,4,0.067358


In [68]:
df_test = pd.read_csv(DATA_DIR + 'test.csv')
df_test.head()

,id,content,lang
0,0,Doctor Who adlı viki başlığına 12. doctor olar...,tr
1,1,"Вполне возможно, но я пока не вижу необходимо...",ru
2,2,"Quindi tu sei uno di quelli conservativi , ...",it
3,3,Malesef gerçekleştirilmedi ancak şöyle bir şey...,tr
4,4,:Resim:Seldabagcan.jpg resminde kaynak sorunu ...,tr


In [69]:
len(df_tst)

63812

In [70]:
df_tst = df_tst.merge(df_test[['id', 'lang']], on='id')
df_tst.head()

,id,toxic,lang
0,0,0.146779,tr
1,1,0.216815,ru
2,2,0.656836,it
3,3,0.065074,tr
4,4,0.067358,tr


In [71]:
len(df_tst)

63812

In [72]:
df_tst.loc[df_tst['lang'].isin(['it']), 'toxic'] = 0.5

In [73]:
# df_tst['lang'].unique()

In [74]:
df_tst[['id', 'toxic']].to_csv('/media/ssd/Downloads/submission_test_it.csv')